In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [2]:
import pandas as pd
import numpy as np
import re

In [37]:
wel_fac = pd.read_csv('./data/서울시사회복지시설.csv')
#dis_fac = pd.read_csv('./data/전국장애시설.csv')
dis_po = pd.read_csv('./data/장애인구수.csv', skiprows=[1, 2, 3])
all_po = pd.read_csv('./data/전체인구수.csv', skiprows=[1, 2])
area = pd.read_csv('./data/서울시영역.csv')
infra = pd.read_csv('./data/집객시설.csv')

dis_po_hang = pd.read_csv('./장애인구/장애인구.csv', skiprows=[1, 2])
code = pd.read_csv('./code/서울시코드정보.csv', encoding = 'cp949')

## 장애인복지관 시설 추출 및 주소 잘못된거 수정

In [55]:
# "장애인복지관"이라는 단어가 포함된 행 추출
wel_fac = wel_fac[wel_fac['시설종류명(시설유형)'].str.contains('장애인복지관')]

In [56]:
wel_fac.columns = ['SGG_NM', 'NAME', 'CATEGORY', 'ADDRESS']

# 괄호와 그 안의 내용을 제거하는 함수 정의
def remove_parentheses(text):
    return re.sub(r'\([^)]*\)', '', text).strip()

# ADDRESS 열에서 괄호를 제거
wel_fac['ADDRESS'] = wel_fac['ADDRESS'].apply(remove_parentheses)

#이상한 행 삭제
wel_fac = wel_fac[wel_fac['NAME'] != '시설테스트1호']
wel_fac = wel_fac[wel_fac['NAME'] != '성분도복지관']

# 인덱스 초기화
wel_fac.reset_index(drop=True, inplace=True)

In [57]:
# 잘못된 주소 변경
data = {
    "NAME": ["구립동대문장애인종합복지관", "서울시립남부장애인종합복지관", "서울시립발달장애인복지관", "마포장애인종합복지관"],
    "ADDRESS": [
        "서울특별시 동대문구 청계천로 521 구립동대문장애인종합복지관",
        "서울특별시 동작구 여의대방로20나길 40 서울시립남부장애인종합복지관",
        "서울특별시 동작구 여의대방로20나길 39 서울시립발달장애인복지관",
        "서울특별시 마포구 성산로4길 35 마포장애인종합복지관"
    ]
}

# 데이터를 DataFrame으로 변환
data = pd.DataFrame(data)

# 주소를 변경하는 함수 정의
def update_address(df):
    address_mapping = {
        "구립동대문장애인종합복지관": "서울특별시 동대문구 청계천로 521 다사랑행복센터",
        "서울시립남부장애인종합복지관": "서울특별시 동작구 여의대방로20나길 40 서울시립남부장애인종합복지관",
        "서울시립발달장애인복지관": "서울특별시 동작구 여의대방로20나길 39 서울시립발달장애인복지관",
        "마포장애인종합복지관": "서울특별시 마포구 성산로4길 35 마포장애인종합복지관"
    }
    
    for name, address in address_mapping.items():
        df.loc[df['NAME'] == name, 'ADDRESS'] = address

    return df

# 주소 업데이트
wel_fac = update_address(wel_fac)

In [58]:
wel_fac.to_csv('장애인복지관.csv', index=False, encoding='utf-8-sig')

## 자치구 경계 데이터 통합

In [ ]:
# 서울시 행정구역 데이터 불러오기
a1 = gpd.read_file('./경계/11110.shp')
a2 = gpd.read_file('./경계/11140.shp')
a3 = gpd.read_file('./경계/11170.shp')
a4 = gpd.read_file('./경계/11200.shp')
a5 = gpd.read_file('./경계/11215.shp')
a6 = gpd.read_file('./경계/11230.shp')
a7 = gpd.read_file('./경계/11260.shp')
a8 = gpd.read_file('./경계/11290.shp')
a9 = gpd.read_file('./경계/11305.shp')
a10 = gpd.read_file('./경계/11320.shp')
a11 = gpd.read_file('./경계/11350.shp')
a12 = gpd.read_file('./경계/11380.shp')
a13 = gpd.read_file('./경계/11410.shp')
a14 = gpd.read_file('./경계/11440.shp')
a15 = gpd.read_file('./경계/11470.shp')
a16 = gpd.read_file('./경계/11500.shp')
a17 = gpd.read_file('./경계/11530.shp')
a18 = gpd.read_file('./경계/11545.shp')
a19 = gpd.read_file('./경계/11560.shp')
a20 = gpd.read_file('./경계/11590.shp')
a21 = gpd.read_file('./경계/11620.shp')
a22 = gpd.read_file('./경계/11650.shp')
a23 = gpd.read_file('./경계/11680.shp')
a24 = gpd.read_file('./경계/11710.shp')
a25 = gpd.read_file('./경계/11740.shp')

In [ ]:
# 모든 GeoDataFrame 병합
seoul_all = pd.concat([a1, a2, a3, a4, a5, a6, a7, a8, a9, a10,
                     a11, a12, a13, a14, a15, a16, a17, a18, a19,
                     a20, a21, a22, a23, a24, a25], ignore_index=True)

# 병합된 GeoDataFrame 저장
seoul_all.to_file('./경계/seoul_all.shp')

## 행정동 경계 데이터 정제

- 용강동은 데이터가 이상해서 제외시킴

In [3]:
# 서울시 행정동 경계 데이터 정제
seuol_dong = gpd.read_file('./법정동경계/LSMD_ADM_SECT_UMD.shp')
seuol_dong = seuol_dong[seuol_dong['COL_ADM_SE'].isin(seoul_all['SGG_CD'])]
seuol_dong = seuol_dong[seuol_dong['EMD_NM'] != '용강동']

seuol_dong.to_file('./법정동경계/seoul_dong.shp')

## 법정동, 행정동 통합

- 경계 데이터는 법정동, 장애인구는 행정동으로 되어있음
- 장애인구의 행정동은 또 이름이 잘못되어 있음. 법정동, 행정동 코드 데이터에서 가져와서 수정

In [38]:
dis_po_h = dis_po_hang.iloc[:,[1,2,3]]
dis_po_h.columns = ['시군구명','EMD_NM','num']

# 행정동에서 기타라고 되어 있는 부분 어차피 인구수가 누락이라 제거
dis_po_h = dis_po_h[dis_po_h['EMD_NM'] != '기타']

# num 열의 타입을 int로 변환
dis_po_h['num'] = dis_po_h['num'].astype(int)

In [39]:
# '제' 글자 제거하는 함수 정의
def remove_je(text):
    if isinstance(text, str):
        return re.sub(r'제(\d+)', r'\1', text)
    return text

In [40]:
# 법정동, 행정동 통합 정보 제공하는 데이터와 장애인구의 행정동명 통합위한 전처리
code['행정동명'] = code['행정동명'].apply(remove_je)

code = code.loc[:,['시군구명','법정동명','행정동명','시군구코드']]

code = code.drop_duplicates(['법정동명','행정동명'])

# 행정동명 값 변경
code['행정동명'] = code['행정동명'].replace({'중계2,3동': '중계2.3동', '보래매동': '보라매동'})

In [41]:
# 데이터프레임 병합
merged_df = pd.merge(dis_po_h, code, 
                     left_on=['시군구명','EMD_NM'], right_on=['시군구명','행정동명'], how='left')

# 불필요한 열 제거
merged_df.drop(columns=['EMD_NM'], inplace=True)

In [42]:
merged_df['행정동명_수'] = merged_df.groupby(['시군구명','행정동명'])['행정동명'].transform('count')
merged_df['num'] = merged_df['num']/merged_df['행정동명_수']

In [53]:
# 법정동면을 기준으로 num의 합계를 계산
result_df = merged_df.groupby(['시군구명','법정동명']).agg({'num': 'sum', '시군구코드': 'first'}).reset_index()
result_df['시군구코드'] = result_df['시군구코드'].astype('str')
result_df.to_csv('./장애인구/장애인구법정동.csv', index=False, encoding='utf-8-sig')

In [47]:
sum(result_df['num'])

389777.9999999996

In [48]:
sum(dis_po_h['num'])

389778